<a href="https://colab.research.google.com/github/0xVolt/whats-up-doc/blob/main/src/experimental-notebooks/code-t5-code-documentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
%pip install -q --no-cache transformers datasets nltk sentencepiece

In [16]:
from transformers import RobertaTokenizer, T5Tokenizer, T5ForConditionalGeneration
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from datasets import load_dataset

In [17]:
def calculateSmoothedBLEU(reference, candidate):
    """
    Calculate the Smoothed BLEU-4 score between a reference string and a candidate string.

    Args:
    reference (str): The reference (ground truth) string.
    candidate (str): The candidate (generated) string.

    Returns:
    float: The Smoothed BLEU-4 score.
    """
    reference_tokens = reference.split()
    candidate_tokens = candidate.split()

    smoother = SmoothingFunction()
    bleu_score = sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=smoother.method1)

    return bleu_score

In [25]:
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base-multi-sum')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base-multi-sum')

# tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
# model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")

In [26]:
data = load_dataset("code_search_net", "python")['test']
data

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 22176
})

In [37]:
testFunction = '''
def calculateSmoothedBLEU(reference, candidate):
    reference_tokens = reference.split()
    candidate_tokens = candidate.split()

    smoother = SmoothingFunction()
    bleu_score = sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=smoother.method1)

    return bleu_score
'''

In [32]:
idx = 1600

text = data['func_code_string'][idx]
label = data['func_documentation_string'][idx]

labelFirstLine = label.split("\n")[0]

print(f"Text:\n{text}\n\nLabel:\n{label}\n\nLabel's First Line:\n{labelFirstLine}")

Text:
def _create_input_order(input_size, input_order="left-to-right"):
  """Returns a degree vectors for the input."""
  if isinstance(input_order, six.string_types):
    if input_order == "left-to-right":
      return np.arange(start=1, stop=input_size + 1)
    elif input_order == "right-to-left":
      return np.arange(start=input_size, stop=0, step=-1)
    elif input_order == "random":
      ret = np.arange(start=1, stop=input_size + 1)
      np.random.shuffle(ret)
      return ret
  elif np.all(np.sort(input_order) == np.arange(1, input_size + 1)):
    return np.array(input_order)

  raise ValueError("Invalid input order: '{}'.".format(input_order))

Label:
Returns a degree vectors for the input.

Label's First Line:
Returns a degree vectors for the input.


In [38]:
input_ids = tokenizer(testFunction, return_tensors="pt").input_ids
generated_ids = model.generate(input_ids, max_length=20)

In [39]:
predicted = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
predicted

'Calculate the BLEU score of two strings.'

In [35]:
calculateSmoothedBLEU(label, predicted)

0.8091067115702212

In [36]:
calculateSmoothedBLEU(labelFirstLine, predicted)

0.8091067115702212